In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from collections import OrderedDict
from time import time

import numpy as np
import scipy as sp
import pandas as pd

from scipy.optimize import fmin_powell
from scipy import integrate
from scipy import linalg

from sklearn.preprocessing import normalize
from sklearn import linear_model
from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

np.set_printoptions(precision=4, suppress=True)

from collections import Counter
from Levenshtein import distance as levenshtein_distance

sns.set_style("whitegrid")
sns.set_palette("colorblind")
palette = sns.color_palette()
figsize = (15,8)
legend_fontsize = 16

from matplotlib import rc
rc('font',**{'family':'sans-serif'})
rc('text', usetex=True)
rc('text.latex',preamble=r'\usepackage[utf8]{inputenc}')
rc('text.latex',preamble=r'\usepackage[russian]{babel}')
rc('axes', **{'titlesize': '16', 'labelsize': '16'})
rc('legend', **{'fontsize': '16'})
rc('figure', **{'dpi' : 200})

## Вариационный вывод в смеси гауссианов

In [ ]:
true_K = 3
mu_arr = np.random.random(true_K) * 20 - 10
sigma_arr = np.random.random(true_K) * 1 + 0.5
print("Средние: %s\nДисперсии: %s" % (mu_arr, sigma_arr))

In [ ]:
each_N = 200
N = true_K * each_N
X = np.hstack( [ np.random.normal(loc=mu_arr[i], scale=sigma_arr[i], size=each_N) for i in range(true_K) ] )

In [ ]:
def sample_from_mixture(k, n, pi, mu, sigma):
    z = np.random.choice(k, n, p=pi)
    return np.random.normal(mu[z], sigma[z])

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
sns.distplot(X, hist=True, color='0.2', kde=True, bins=100, label="Эмпирическая плотность")
sns.distplot(sample_from_mixture(true_K, 200*each_N, np.ones(true_K) / float(true_K), mu_arr, sigma_arr), hist=False, kde=True, label="Плотность смеси")
ax.set_ylabel("Плотность распределения")
ax.legend()
# fig.savefig('varmixture1.pdf', bbox_inches='tight')

In [ ]:
## обновляем r_nk в вариационном выводе
def update_r(X, alpha, beta, m, a, b):
    rho = np.zeros((N, K))
    rho = rho + sp.special.digamma(alpha) - sp.special.digamma(np.sum(alpha)) \
        + .5 * (sp.special.digamma(a) - np.log(b)) \
        - .5 * np.log(2*np.pi) \
        - .5 / beta - a * (X[:,np.newaxis] - m)**2 / (2*b)
    r = np.exp(rho - np.logaddexp.reduce(rho, axis=1)[:,np.newaxis])
    return r

## обновляем параметры в вариационном выводе
def update_params(X, r, alpha, beta, m, a, b):
    N_k = np.sum(r, axis=0)
    Sum_k = np.sum(r * X[:,np.newaxis], axis=0)
    Sumsq_k = np.sum(r * (X[:,np.newaxis] ** 2), axis=0)
    new_alpha = alpha + N_k
    new_beta = beta + N_k
    new_m = ( beta * m + Sum_k ) / new_beta
    new_a = a + .5 * N_k
    new_b = b + .5 * (beta * (m**2) + Sumsq_k - new_beta * (new_m**2) )
    return new_alpha, new_beta, new_m, new_a, new_b

In [ ]:
print("Истинные средние:\t%s\n" % mu_arr)

K = 3

## Инициализируем параметры модели
alpha_0 = np.ones(K) * 0.01
alpha = alpha_0
beta = np.ones(K)
m = np.random.random(K) * (X.max() - X.min()) + X.min()
a = np.ones(K)
b = np.ones(K)

param_history = [(alpha, beta, m, a, b)]

print("Начальные значения:\tm = %s\tbeta = %s\ta = %s\tb = %s" % (m, beta, a, b))

eps = 0.01
for nIter in range(10000):
    r = update_r(X, alpha, beta, m, a, b)
    alpha, beta, m, a, b = update_params(X, r, alpha, beta, m, a, b)
    param_history.append((alpha, beta, m, a, b))
#     elbo_values.append(0) # elbo(X, m, s2, r))
    if nIter % 1000 == 0:
        print("\t%d:\tm = %s\talpha = %s" % (nIter, m, alpha))
#     if np.abs(elbo_values[-2] - elbo_values[-1]) <= eps:
#         break

print("\nПосле %d итераций:\tm = %s\talpha = %s" % (nIter, m, alpha))

In [ ]:
print("alpha/sum(alpha) = %s" % (alpha / np.sum(alpha)))
print("m = %s" % m)
print("a = %s" % a)
print("b = %s" % b)
print("sqrt(b/a) = %s" % (1 / np.sqrt(a / b)))

In [ ]:
## ожидаемый вес в каждом кластере
expected_pi = alpha / np.sum(alpha)
expected_pi

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
sns.distplot(X, hist=True, color='0.2', kde=True, bins=100, label="Эмпирическая плотность")
sns.distplot(sample_from_mixture(true_K, 50*each_N, np.ones(true_K) / float(true_K), mu_arr, np.ones(true_K)), hist=False, kde=True, label="Плотность смеси")
sns.distplot(sample_from_mixture(K, 50*each_N, expected_pi, m, 1 / np.sqrt(a/b)), hist=False, kde=True, label="Апостериорная плотность")
ax.set_ylabel("Плотность распределения")
ax.legend()
# fig.savefig('varmixture2.pdf', bbox_inches='tight')

In [ ]:
print("Истинные средние:\t%s\n" % mu_arr)

K = 6

## Инициализируем параметры модели
alpha_0 = np.ones(K) * 0.0001
alpha = alpha_0
beta = np.ones(K)
m = np.random.random(K) * (X.max() - X.min()) + X.min()
a = np.ones(K)
b = np.ones(K)

param_history = [(alpha, beta, m, a, b)]

print("Начальные значения:\tm = %s\tbeta = %s" % (m, beta))

eps = 0.01
for nIter in range(10000):
    r = update_r(X, alpha, beta, m, a, b)
    alpha, beta, m, a, b = update_params(X, r, alpha, beta, m, a, b)
    param_history.append((alpha, beta, m, a, b))
    if nIter % 1000 == 0:
        print("\t%d:\tm = %s\talpha = %s" % (nIter, m, alpha / sum(alpha)))
#     if np.abs(elbo_values[-2] - elbo_values[-1]) <= eps:
#         break

print("\nПосле %d итераций:\tm = %s\talpha = %s" % (nIter, m, alpha / sum(alpha)))

In [ ]:
print("alpha/sum(alpha) = %s" % (alpha / np.sum(alpha)))
print("m = %s" % m)
print("a = %s" % a)
print("b = %s" % b)
print("sqrt(b/a) = %s" % (1 / np.sqrt(a / b)))

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
sns.distplot(X, hist=True, color='0.2', kde=True, bins=100, label="Эмпирическая плотность")
sns.distplot(sample_from_mixture(true_K, 50*each_N, np.ones(true_K) / float(true_K), mu_arr, np.ones(true_K)), hist=False, kde=True, label="Плотность смеси")
sns.distplot(sample_from_mixture(K, 50*each_N, (alpha / np.sum(alpha)), m, 1 / np.sqrt(a/b)), hist=False, kde=True, label="Апостериорная плотность")
ax.set_ylabel("Плотность распределения")
ax.legend()
# fig.savefig('varmixture3.pdf', bbox_inches='tight')